In [ ]:
%%shell
%%shell
# Update package lists
apt-get update

# Install essential build tools (if not already present, good practice)
apt-get install -y build-essential

# Install OpenCV development libraries (C++ headers and shared libraries)
# 'libopencv-dev' provides the development files
# 'python3-opencv' is for Python, but sometimes pulled in as a dependency
apt-get install -y libopencv-dev python3-opencv

# Verify OpenCV version (optional, but good for checking)
pkg-config --modversion opencv4 # For OpenCV 4.x
# Or for older versions: pkg-config --modversion opencv

/bin/bash: line 1: fg: no job control
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,853 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,147 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,160 kB]


In [ ]:
%%writefile opencv_example.cpp
#include <opencv2/opencv.hpp> // Main OpenCV header
#include <iostream>

int main() {
    // Define input and output file names
    std::string input_filename = "my_image.jpg"; // Change this to your uploaded image name
    std::string output_filename = "my_image_grayscale.jpg";

    // Read the image from disk
    // cv::imread returns an empty Mat object if the image cannot be read
    cv::Mat image = cv::imread(input_filename, cv::IMREAD_COLOR);

    // Check if the image was loaded successfully
    if (image.empty()) {
        std::cerr << "Error: Could not open or find the image " << input_filename << std::endl;
        return -1; // Indicate an error
    }

    std::cout << "Image '" << input_filename << "' loaded successfully. Dimensions: "
              << image.cols << "x" << image.rows << std::endl;

    // Convert the image to grayscale
    cv::Mat grayscale_image;
    cv::cvtColor(image, grayscale_image, cv::COLOR_BGR2GRAY);

    std::cout << "Image converted to grayscale." << std::endl;

    // Save the grayscale image
    bool success = cv::imwrite(output_filename, grayscale_image);

    if (success) {
        std::cout << "Grayscale image saved as '" << output_filename << "'" << std::endl;
    } else {
        std::cerr << "Error: Could not save the grayscale image." << std::endl;
        return -1;
    }

    return 0; // Indicate success
}

Writing opencv_example.cpp


In [ ]:
%%shell
# Compile the C++ code
# `pkg-config --cflags opencv4` gives include paths (-I)
# `pkg-config --libs opencv4` gives library paths (-L) and library names (-l)
g++ opencv_example.cpp -o opencv_app $(pkg-config --cflags --libs opencv4)

# Check if compilation was successful (optional, but good practice)
if [ -f opencv_app ]; then
    echo "Compilation successful. Executable 'opencv_app' created."
else
    echo "Compilation FAILED. Check for errors above."
fi

Compilation successful. Executable 'opencv_app' created.


In [ ]:
%%shell
./opencv_app

Image 'my_image.jpg' loaded successfully. Dimensions: 382x509
Image converted to grayscale.
Grayscale image saved as 'my_image_grayscale.jpg'


In [ ]:
!pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp59n5i67m".


In [ ]:
%%shell
nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
%%shell
nvidia-smi

Sun Jul 27 12:15:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%%writefile hello.cu
#include <iostream>
#include "cuda_runtime.h"

__global__ void hello_cuda(){
    printf("Hello from CUDA GPU!\\n"); //This won't be printed. Just to let know that printf will not work in CUDA kernel
}

int main(){
    std::cout<<"Starting CUDA kernel...\n";
    hello_cuda<<<1,1>>>();
    cudaDeviceSynchronize();
    std::cout<<"Kernel Execution finished.\n";
    return 0;
}

Writing hello.cu


In [ ]:
%%shell
ls -l hello.cu

-rw-r--r-- 1 root root 294 Jul 27 11:32 hello.cu


In [ ]:
%%shell
nvcc hello.cu -o hello_cuda 2>&1 | tee nvcc_output.txt
cat nvcc_output.txt

In [ ]:
%%shell
ls -l hello_cuda

-rwxr-xr-x 1 root root 1003624 Jul 27 11:33 hello_cuda


In [ ]:
%%shell
./hello_cuda 2>&1 | tee cuda_program_output.txt
cat cuda_program_output.txt

Starting CUDA kernel...
Kernel Execution finished.
Starting CUDA kernel...
Kernel Execution finished.


In [ ]:
%%shell
./hello_cuda

Starting CUDA kernel...
Kernel Execution finished.


In [ ]:
%%writefile add.cu

#include <iostream>
#include <math.h>
// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20
 ;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 1>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Writing add.cu


In [ ]:
%%shell
nvcc add.cu -o add_numbers_executable
./add_numbers_executable

Max error: 1


In [ ]:
%%shell
pwd

/content
